In [1]:
from random import choice, choices
import base64
from Crypto.Cipher import AES

BLOCKSIZE = 16

In [2]:
def XOR(A: bytes, B: bytes):
    assert type(A) is bytes and type(B) is bytes and len(A) == len(B)
    return bytes([ a^b for a, b in zip(A, B) ])

def IntToLittleEndian(x: int, nbytes: int):
    assert type(x) is int and x >= 0
    out = bytes()
    while x >  0:
        out += bytes([ x & 0xFF ])
        x >>= 8
    
    assert len(out) <= nbytes
    out += bytes( [0] * ( nbytes-len(out) ) )
    
    return out

def LittleEndianToInt(x: bytes):
    assert type(x) is bytes
    out = 0
    byte_counter = 0
    while len(x) > 0:
        out += x[0] << byte_counter*8
        byte_counter += 1
        x = x[1:]
        
    return out

In [3]:
class CTR(object):
    def __init__(self, nonce: bytes, key: bytes):
        assert type(nonce) is bytes and len(nonce) < BLOCKSIZE
        self.nonce = nonce
        self.aes = AES.new(key, AES.MODE_ECB)
        
    def EncryptBlock(self, block: bytes, counter: int):
        assert type(counter) is int and counter >= 0
        assert type(block) is bytes and len(block) == BLOCKSIZE
        little_endian: bytes = IntToLittleEndian(counter, nbytes = BLOCKSIZE-len(self.nonce))
        key_block = self.aes.encrypt( self.nonce + little_endian )
        assert type(key_block) is bytes and len(key_block) == BLOCKSIZE
        return XOR(key_block, block)
    
    def DecryptBlock(self, block, counter: int):
        return self.EncryptBlock(block, counter)
    
    def EncryptBytes(self, plaintext: bytes, counter: int):
        assert type(plaintext) is bytes and len(plaintext) <= BLOCKSIZE
        assert type(counter) is int and counter >= 0
        little_endian: bytes = IntToLittleEndian(counter, nbytes = BLOCKSIZE-len(self.nonce))
        key_block = self.aes.encrypt( self.nonce + little_endian )
        assert type(key_block) is bytes and len(key_block) == BLOCKSIZE
        return XOR(key_block[:len(plaintext)], plaintext)
    
    def DecryptBytes(self, ciphertext: bytes, counter: int):
        return self.EncryptBytes(ciphertext, counter)
    
    def EncryptStream(self, plaintext: bytes, counter: int = 0):
        assert type(plaintext) is bytes
        stream = bytes(plaintext)
        ciphertexts = list()
        
        while len(stream) > 0:
            block =  stream[:BLOCKSIZE]
            stream = stream[BLOCKSIZE:]
            assert len(block) == BLOCKSIZE or len(stream) == 0
            ciphertexts += [ self.EncryptBytes(block, counter) ]
            counter += 1
        
        ciphertext = b''.join(ciphertexts)
        assert len(ciphertext) == len(plaintext)
        return ciphertext
    
    def DecryptStream(self, plaintext: bytes, counter: int = 0):
        return self.EncryptStream(plaintext, counter)
    
        

In [4]:
ctr_cipher = CTR(nonce = b'\0'*8, key = b'YELLOW SUBMARINE')
ciphertexts = [ ctr_cipher.EncryptStream(base64.decodebytes(line).lower()) for line in open('strings.txt', mode = 'rb') ]
for ciphertext in ciphertexts:
    print(ciphertext)

b'\x1f\xf1\xa3*\xd9\xc7f\x8f\x86\xdb#)\x04v\xaeR\xb3\x98L\xbf\xf4\x02a\xbb\xef\xb5y\xb3\xcb\x8f\n'
b'\x15\xbe\xa6"\xc1\xc5f\x95\x8a\xdbk}\x1az\xb5\x1b\xb6\xcc\n\xbd\xfb\x08a'
b'\x10\xa3\xa4&\x8f\xc1)\x97\x8d\xdbf/L|\xb1R\xb6\x89\x1f\xb7\xb8\x0c\x7f\xb1\xa1\xbd?\xf4\xdd\x8b\n'
b'\x13\xb8\xac#\xdb\xc7#\x8c\x97\xc7.>\t}\xb7\x07\xa0\x95L\xb4\xf7\x18a\xbb\xbc\xf4'
b'\x1f\xf1\xa3*\xd9\xc7f\x92\x82\xdcp8\x083\xb4\x1b\xa6\x84L\xbd\xb8\x03}\xba\xef\xb5y\xb3\xdb\x86\x168E\xc5\xef\xaf'
b'\x19\xa3\xeb;\xc0\xce/\x96\x86\x8fn8\r}\xaa\x1c\xb5\x80\t\xaf\xebMe\xb1\xbd\xbel\xbf'
b'\x19\xa3\xeb#\xce\xd4#\xc2\x8f\xc6m:\ta\xa6\x16\xf2\x8d\x1b\xb4\xf1\x01w\xfe\xae\xb4{\xb3\xdc\x8f\x1a|'
b'\x06\xbe\xa7"\xdb\xc7f\x8f\x86\xcem4\x02t\xaf\x17\xa1\x9fL\xab\xf7\x1fv\xad\xe3'
b'\x17\xbf\xafk\xdb\xca)\x97\x84\xc7w}\x0ev\xa5\x1d\xa0\x89L\xb5\xb8\x05s\xba\xef\xbep\xfd\xca'
b'\x19\xb7\xeb*\x8f\xcf)\x81\x88\xc6m:Lg\xa2\x1e\xb7\xcc\x03\xae\xb8\x0c2\xb9\xa6\xb8z'
b"\x02\xbe\xeb;\xc3\xc7'\x91\x86\x8fb}\x0f|\xae\x02\xb3\x82

In [5]:
# Compute some statistics on token frequency
paradise = open('paradise.txt', mode =  'rb').read().lower()
Counter = lambda x, normalize = False: { key: x.count(key) / ( len(x) if normalize else 1 ) for key in {*x} }
alphabet = b'abcdefghijklmnopqrstuvwxyz'
base_frequency = Counter([ char for char in paradise if char in alphabet ], normalize = True)

In [39]:
# This scoring function worked well in challenge 6 so I'll use it again here

def Dot(x, y):
    assert len(x) == len(y)
    return sum( x_*y_ for x_, y_ in zip(x, y) )

def Norm(x):
    return sum( x_**2 for x_ in x ) ** 0.5

def Mean(x):
    return sum(x) / len(x)

def Score(x: bytes, alpha = 6):
    assert type(x) is bytes
    x = x.lower()
  
    frequencies = 0.082, 0.015, 0.028, 0.043, 0.130, 0.022, 0.020, 0.061, 0.070, 0.002, 0.008, 0.040, 0.024, \
                  0.067, 0.075, 0.019, 0.001, 0.060, 0.063, 0.091, 0.028, 0.010, 0.024, 0.002, 0.020, 0.001
    alphabet = b'abcdefghijklmnopqrstuvwxyz'
    
    v1 = [0] * 256
    for n, char in enumerate(alphabet):
        v1[char] = frequencies[n]
    assert 0.99 < sum(v1) < 1.01
    
    v2 = [0] * 256
    for char in range(256):
        v2[char] = x.count(char) / len(x)
    assert 0.99 < sum(v2) < 1.01

    return Dot(v1, v2) * ( sum( char in { *b' \n', *range(97, 123) } for char in x ) / len(x) ) ** alpha

In [27]:
Score(b"Before all Temples th' upright heart and pure,")

0.04070896872617271

In [31]:
Score(b'qqqqqqqqqqqqq')

0.001

In [37]:
solved_keys = list()

for idx in range( max( len(ciphertext) for ciphertext in ciphertexts ) ):
    text = bytes( ciphertext[idx] for ciphertext in ciphertexts if len(ciphertext) > idx )
    best_score = 0
    best_key = -1
    
    for key in range(0xFF):
        plaintext = bytes([ key^char for char in text ])
        score = Score(plaintext)
        if score > best_score:
            best_score = score
            best_key = key
    
    solved_keys.append(best_key)

In [38]:
for ciphertext in ciphertexts:
    print(XOR(ciphertext, bytes(solved_keys[:len(ciphertext)])))

b'I have met them at close of daY'
b'Coming with vivid faces'
b'From counter or desk among greY'
b'Eighteenth-century houses.'
b'I have passed with a nod of thE hCee'
b'Or polite meaningless words,'
b'Or have lingered awhile and saId'
b'Polite meaningless words,'
b'And thought before i had done'
b'Of a mocking tale or a gibe'
b'To please a companion'
b'Around the fire at the club,'
b'Being certain that they and i'
b'But lived where motley is worn\x1a'
b'All changed, changed utterly:'
b'A terrible beauty is born.'
b"That woman's days were spent"
b'In ignorant good will,'
b'Her nights in argument'
b'Until her voice grew shrill.'
b'What voice more sweet than herS'
b'When young and beautiful,'
b'She rode to harriers?'
b'This man had kept a school'
b'And rode our winged horse.'
b'This other his helper and frieNd'
b'Was coming into his force;'
b'He might have won fame in the End\n'
b'So sensitive his nature seemed\x0c'
b'So daring and sweet his thoughT.'
b'This other man i had dreamed'
b'A d